In [2]:
import scipy
import scipy.io
import numpy as np
from scipy.linalg import norm
from scipy.sparse import csr_matrix
from scipy.linalg import norm
import pickle
import time
from collections import defaultdict
import json
from sklearn.datasets import load_svmlight_file
import matplotlib.pyplot as plt
from line_profiler import LineProfiler

$$\min_{x \in \mathbb R^n_+; \|x\|_1 \le M} f(x) = \sum_{i = 1}^N w_i^T x - \sum_{i = 1}^N y_i \ln (w_i^T x) + \lambda \|x\|_1$$

In [3]:
def poisson(W, y, x, lam):
    """
        W -- object matrix (N x n)
        y -- labels (N)
        x -- weights (n)
        lam -- regularization param
    """ 
    dot_product = W @ x # N
#     print(x)
#     print(dot_product, end='\n\n')
    fst_term = np.sum(dot_product)
    snd_term = np.sum(y * np.log(dot_product))
    return fst_term - snd_term + lam * np.abs(x).sum(), dot_product

$$\nabla f(x) = \sum_{i=1}^N w_i - \sum_{i = 1}^N y_i \frac{w_i}{w_i^T x} + \lambda \cdot sign(x)$$

In [4]:
def fn_Poisson_Grad(x, A, b, lam):
    Btm = A.dot(x)  # Btm(i) = a_i^T x
    Par = 1 - b / Btm
    i = list(range(N))
    DPar = scipy.sparse.csr_matrix((Par, (i, i)), shape=(N, N))
    ResM = DPar.dot(A)
    l1_grad = np.array([1 if c >=0 else -1 for c in x])
    Grad = np.array(np.sum(ResM, axis=0)).squeeze() + lam * l1_grad
    return Grad, Btm

In [5]:
def grad_poisson(W, y, x, lam, dot_product):
    """
        W -- object matrix (N x n)
        y -- labels (N)
        x -- weights (n)
        lam -- regularization param
        btm -- W @ x (N)
    """
    dot_product = W @ x # N
    fst_term = W.sum(axis=0) # n
    snd_term = y @ (W / dot_product.reshape(-1, 1)) # n
    return fst_term - snd_term + lam * np.sign(x)

$$\nabla^2 f(x) = \sum_{i = 1}^N y_i \frac{w_i w_i^T}{(w_i^T x)^2}$$

In [6]:
def fn_Poisson_Hinv(A, b, lam, Btmv):
    Rat = b / (Btmv)**2
    i = range(N)
    DRat = scipy.sparse.csr_matrix((Rat, (i, i)), shape=(N, N))
    Res1 = (A.T.dot(DRat)).dot(A)
    H = Res1
    H = H.A
    return H

In [7]:
def hess_poisson(W, y, lam, Btm):
    """
        W -- object matrix (N x n)
        y -- labels (N)
        x -- weights (n)
    """
    denom = 1 / (Btm) # N
    snd_einsum = np.multiply(W, denom.reshape(-1, 1))
    fst_einsum = y.reshape(-1, 1) * snd_einsum
    return np.einsum('ij,ik->jk', fst_einsum, snd_einsum)

In [8]:
# alpha policies
def alpha_standard(k):
    return 2 / (k + 2)

def alpha_icml(k, Gap, hess, s, x):
    e = hess.dot((s - x).T).T.dot((s - x).T) ** 0.5
    t = Gap / (e * (Gap + e))
    beta = alpha_standard(k)
    return min(beta, t)

In [ ]:
# N = 10000
# n = 500
# x_foo = np.random.rand(n)
# W_foo = np.random.rand(N, n)
# y_foo = np.random.rand(N)
# %time _ = hess_poisson_kam(W_foo, y_foo, 0, W_foo.dot(x_foo))

In [26]:
def frank_wolfe_poisson(W,
                        y,
                        lam,
                        x_0=None,
                        alpha_policy='standard',
                        max_iter=50000,
                        eps=0.001,
                        print_every=100,
                        dtype=np.float64,
                        debug_info=False):
    """
        W -- object matrix (N x n)
        y -- labels (N)
        lam -- regularization param
        x_0 -- starting point (n)
        alpha_policy -- name of alpha changing policy function
        eps -- epsilon
        print_every -- print results every print_every steps
    """
    int_start = time.time()
    x_0 = np.ones(W.shape[1]) if x_0 is None else x_0
    W = W.astype(dtype)
    x_0 = x_0.astype(dtype)
    
    x = x_0
    x_hist = []
    alpha_hist = []
    Gap_hist = []
    Q_hist = []
    time_hist = []
    grad_hist = []
    print('********* Algorithm starts *********')
    for k in range(1, max_iter + 1):
        start_time = time.time()
        Q, dot_product = poisson(W, y, x, lam)

        grad_f, Btm = fn_Poisson_Grad(x, W, y, lam)
        #grad_f = grad_poisson(W, y, x, lam, dot_product) # 1 x n
        
        i_max = np.argmax(np.abs(grad_f))
        s = np.zeros(n)
        s[i_max] = np.sign(-grad_f[i_max]) # 1 x n
        Gap = grad_f @ (x - s)
        
        if alpha_policy == 'standard':
            alpha = alpha_standard(k)
        elif alpha_policy == 'icml':
            hessian = fn_Poisson_Hinv(W, y, lam, dot_product)
            #hessian = hess_poisson(W, y, lam, dot_product)
            alpha = alpha_icml(k, Gap, hessian, s, x)
        alpha = dtype(alpha)
            
        # filling history
        x_hist.append(x)
        alpha_hist.append(alpha)
        Gap_hist.append(Gap)
        Q_hist.append(Q)
        grad_hist.append(grad_f)
        
        x = x + alpha * (s - x)
        
        criterion = norm(x - x_hist[-1]) / max(1, norm(x_hist[-1]))
#         criterion = Gap
        if criterion <= eps:
            int_end = time.time()
            print('Convergence achieved!')
            print(f'iter = {k}, stepsize = {alpha}, crit = {criterion}, Q={Q}')
            print('Total time: %f' % (int_end - int_start))
            return x, alpha_hist, Gap_hist, Q_hist, time_hist, grad_hist
            
        if k % print_every == 0 or k == 1:
            if not debug_info:
                print(f'iter = {k}, stepsize = {alpha}, criterion = {criterion}, Q={Q}')
            else:
                print(k)
                print(f's = {np.nonzero(s)}')
                print(f'Gap = {Gap}')
                print(f'alpha = {alpha}')
                print(f'criterion = {criterion}')
                print(f'grad = {grad_f}')
                print(f'grad norm = {norm(grad_f)}')
                print(f'x = {x}')
#                 x_nz = x[np.nonzero(x)[0]]
#                 print(f'x non zero: {list(zip(x_nz, np.nonzero(x)[0]))}\n')
        time_hist.append(time.time() - start_time)
    int_end = time.time()
    print('Total time: %f' % (int_end - int_start))
    if alpha_policy == 'icml':
        return x, alpha_hist, Gap_hist, Q_hist, time_hist, grad_hist
    return x, alpha_hist, Gap_hist, Q_hist, time_hist, grad_hist

## Data load

In [37]:
W, y = load_svmlight_file('./data/a9a')
y = y.reshape(-1, 1)
W = W.toarray()
N, n = W.shape
lam = 1 / 2 * np.sqrt(N)

In [38]:
W.shape, y.shape

((32561, 123), (32561, 1))

In [39]:
#y = y.squeeze()

### Radius

In [40]:
R = 1
W /= R

### Some preprocessing

In [41]:
import numpy.matlib

In [42]:
# поэлементное умножение ответа от объекта на все признаки объекта
W = numpy.matlib.repmat(y, 1, W.shape[1]) * W
# суммирование признаковых описаний
sA = np.sum(W, 1)
# если сумма строки отрицательна, её инвертирование
W[sA < 0, :] = -W[sA < 0, :]
# удаление строк с нулевой суммой
W = W[sA != 0, :]
W = scipy.sparse.csr_matrix(W)

In [43]:
Bias = 1
b = (Bias * y).squeeze()
b = np.abs(b)
if (b.any == 0):
    print(' Input parameter y error')

In [44]:
y = b
y.shape

(32561,)

## Standard

In [45]:
lp = LineProfiler()
lp_wrapper = lp(frank_wolfe_poisson)
x, \
alpha_hist, \
Gap_hist, \
Q_hist, \
time_hist, \
grad_hist = lp_wrapper(W, y, lam,x_0=np.ones(n) / n,
                       eps=1e-4, max_iter=50000, 
                       alpha_policy='standard', 
                       print_every=100, 
                       debug_info=True)

********* Algorithm starts *********
1
s = (array([75], dtype=int64),)
Gap = 215778.92897541937
alpha = 0.6666666666666666
criterion = 0.663951108875809
grad = [-5.08552577e+04 -4.60356346e+04 -5.34270049e+04 -4.98588139e+04
 -5.59768918e+04 -1.76891436e+05 -1.97216613e+04 -8.62701842e+03
 -7.39555139e+03 -1.62228481e+04 -1.00284745e+04 -1.87766660e+01
  3.09925648e+01 -5.12015474e+04 -5.12708571e+04 -5.12244880e+04
 -5.12292494e+04 -5.12274610e+04 -4.19553846e+04 -5.75014917e+04
 -9.23685908e+03 -8.25384028e+04 -4.43245024e+03 -8.29406513e+03
 -1.07702804e+04 -3.99171897e+03 -5.05906113e+03 -3.35037557e+03
 -1.34282195e+04 -1.23872722e+03 -7.32999420e+03 -3.16278491e+03
 -2.55648546e+03 -3.14845347e+02 -3.37096303e+04 -8.25384028e+04
 -5.75014917e+04 -1.91545689e+04 -6.32495092e+04 -1.17635089e+05
 -3.48062050e+04 -8.43436740e+04 -8.00029315e+03 -7.87474420e+03
 -3.22137382e+03 -9.17766660e+01 -7.14573271e+03 -3.18700514e+04
 -2.56197986e+04 -2.83722382e+04 -3.16165019e+04 -3.22115679

300
s = (array([82], dtype=int64),)
Gap = 15.169420238193766
alpha = 0.006622516556291391
criterion = 0.006509102982269226
grad = [-4.26086120e+02 -6.01320954e+02 -8.18475303e+02 -8.37498709e+02
 -9.16798207e+02 -2.61408853e+03 -2.41119565e+02 -1.12627118e+02
 -5.97199019e+01 -1.99227906e+02 -6.98296436e+01  8.87769082e+01
  8.99271248e+01 -6.25151422e+02 -7.97306492e+02 -7.03046057e+02
 -7.01577158e+02 -7.73098165e+02 -7.32801075e+02 -6.71287021e+02
 -2.11122237e+01 -9.80348176e+02 -6.14110876e+01 -3.91731460e+01
 -6.49803824e+01  2.58937663e+01  2.21371681e+00  4.39349914e+01
 -2.35858607e+02  5.23795972e+01  4.99730304e+00 -1.90032884e+01
  1.19049027e+01  7.69281113e+01 -4.34423173e+02 -9.80348176e+02
 -6.71287021e+02 -1.94376862e+02 -1.31974406e+03 -2.08895767e+03
 -3.75318383e+02 -9.72499250e+02 -3.97105369e+01 -3.90502772e+01
  6.87732620e+00  8.89261605e+01 -3.48794250e+01 -3.45124867e+02
 -2.71961798e+02 -3.46051044e+02 -5.40881637e+02 -6.05329767e+02
 -3.78399167e+01 -1.54326

600
s = (array([82], dtype=int64),)
Gap = 7.280079201346325
alpha = 0.0033222591362126247
criterion = 0.0032602087197038973
grad = [-4.14426066e+02 -5.94931372e+02 -8.13321509e+02 -8.32814081e+02
 -9.13011236e+02 -2.58789667e+03 -2.42108834e+02 -1.13943643e+02
 -5.89538319e+01 -1.96521877e+02 -6.84155792e+01  8.87827703e+01
  8.99362131e+01 -6.17584914e+02 -7.90938144e+02 -6.96179364e+02
 -6.95553668e+02 -7.68248172e+02 -7.29520191e+02 -6.60800523e+02
 -1.95089153e+01 -9.68217637e+02 -6.21477065e+01 -3.76298825e+01
 -6.32661005e+01  2.61473522e+01  2.28410022e+00  4.43552127e+01
 -2.35038973e+02  5.21872306e+01  6.17408225e+00 -1.94139877e+01
  1.14796902e+01  7.68686595e+01 -4.31575926e+02 -9.68217637e+02
 -6.60800523e+02 -1.91119317e+02 -1.31679086e+03 -2.09591516e+03
 -3.63511606e+02 -9.52882089e+02 -3.68603859e+01 -3.52230139e+01
  7.35129017e+00  8.89833679e+01 -3.36267615e+01 -3.47113166e+02
 -2.64673872e+02 -3.41946822e+02 -5.38931137e+02 -6.01671228e+02
 -3.78427872e+01 -1.5284

900
s = (array([73], dtype=int64),)
Gap = 2.694991002555298
alpha = 0.0022172949002217295
criterion = 0.0018849399137017774
grad = [-4.17678612e+02 -5.97166093e+02 -8.13475737e+02 -8.31468423e+02
 -9.08563008e+02 -2.60482263e+03 -2.36757296e+02 -1.11020581e+02
 -5.75884214e+01 -1.93236810e+02 -6.65311614e+01  8.88071969e+01
  8.99421500e+01 -6.16434381e+02 -7.91072178e+02 -6.95790875e+02
 -6.96077042e+02 -7.68977396e+02 -7.28225894e+02 -6.61512146e+02
 -2.00144937e+01 -9.69387344e+02 -6.12557041e+01 -3.76435111e+01
 -6.31702452e+01  2.58902146e+01  2.21249460e+00  4.41347045e+01
 -2.33686215e+02  5.19810519e+01  5.88686008e+00 -1.89329980e+01
  1.10487686e+01  7.67792577e+01 -4.33644480e+02 -9.69387344e+02
 -6.61512146e+02 -1.91037090e+02 -1.31277081e+03 -2.08668972e+03
 -3.64821430e+02 -9.58637385e+02 -3.78891286e+01 -3.56214961e+01
  6.76267374e+00  8.89912865e+01 -3.39219874e+01 -3.46059391e+02
 -2.68214242e+02 -3.42244759e+02 -5.36740355e+02 -5.99433206e+02
 -3.85743870e+01 -1.5449

1200
s = (array([75], dtype=int64),)
Gap = 2.481743650828548
alpha = 0.0016638935108153079
criterion = 0.0011269942450617605
grad = [-4.18085678e+02 -5.97062541e+02 -8.14084012e+02 -8.32602538e+02
 -9.11397558e+02 -2.60224401e+03 -2.38969453e+02 -1.12239761e+02
 -5.79595752e+01 -1.94776065e+02 -6.73503046e+01  8.87958326e+01
  8.99415803e+01 -6.17892297e+02 -7.91810494e+02 -6.97378632e+02
 -6.97009348e+02 -7.69141558e+02 -7.29493119e+02 -6.62490901e+02
 -1.99499824e+01 -9.70049700e+02 -6.15866023e+01 -3.78600944e+01
 -6.35251790e+01  2.59279720e+01  2.11621911e+00  4.41954028e+01
 -2.34572488e+02  5.19395209e+01  5.93842794e+00 -1.91591513e+01
  1.10085059e+01  7.67855154e+01 -4.33601756e+02 -9.70049700e+02
 -6.62490901e+02 -1.91608607e+02 -1.31548136e+03 -2.08986148e+03
 -3.66232047e+02 -9.58606156e+02 -3.77263225e+01 -3.61983456e+01
  6.86404461e+00  8.89746513e+01 -3.39694394e+01 -3.46132306e+02
 -2.67670248e+02 -3.42845732e+02 -5.38225287e+02 -6.01362233e+02
 -3.81094047e+01 -1.539

...

12300
s = (array([75], dtype=int64),)
Gap = 0.26966243404024226
alpha = 0.00016257519102584944
criterion = 0.00011002188148921713
grad = [-4.14786557e+02 -5.95287538e+02 -8.12126914e+02 -8.30557984e+02
 -9.08357112e+02 -2.59785929e+03 -2.37458889e+02 -1.11622632e+02
 -5.73015792e+01 -1.92833194e+02 -6.62315848e+01  8.88082874e+01
  8.99462708e+01 -6.14927458e+02 -7.89520595e+02 -6.94637954e+02
 -6.94669250e+02 -7.67360848e+02 -7.27760249e+02 -6.59087938e+02
 -1.95238238e+01 -9.66236738e+02 -6.15117939e+01 -3.73379418e+01
 -6.28654205e+01  2.60497402e+01  2.25127142e+00  4.43233519e+01
 -2.33822534e+02  5.19506356e+01  6.25943162e+00 -1.91132973e+01
  1.09884934e+01  7.67773793e+01 -4.32486858e+02 -9.66236738e+02
 -6.59087938e+02 -1.90426696e+02 -1.31287788e+03 -2.08888849e+03
 -3.62545639e+02 -9.53276028e+02 -3.70203125e+01 -3.49385661e+01
  7.00079989e+00  8.89987998e+01 -3.36455953e+01 -3.46414493e+02
 -2.65934000e+02 -3.41555046e+02 -5.36877266e+02 -5.99217683e+02
 -3.82640212e+01 -

12600
s = (array([75], dtype=int64),)
Gap = 0.38085992313335737
alpha = 0.00015870496746548166
criterion = 0.00010740673403596178
grad = [-4.15001159e+02 -5.95387328e+02 -8.12303585e+02 -8.30781993e+02
 -9.08720039e+02 -2.59783676e+03 -2.37700690e+02 -1.11730571e+02
 -5.74094848e+01 -1.93099870e+02 -6.63854985e+01  8.88065933e+01
  8.99455227e+01 -6.15244104e+02 -7.89732916e+02 -6.94874110e+02
 -6.94838725e+02 -7.67504249e+02 -7.27932310e+02 -6.59403479e+02
 -1.95551918e+01 -9.66603958e+02 -6.15357850e+01 -3.73869412e+01
 -6.29285747e+01  2.60511103e+01  2.25505640e+00  4.43170866e+01
 -2.33912260e+02  5.19707417e+01  6.23137989e+00 -1.91198413e+01
  1.10307805e+01  7.67847554e+01 -4.32477619e+02 -9.66603958e+02
 -6.59403479e+02 -1.90538850e+02 -1.31317020e+03 -2.08914632e+03
 -3.62856403e+02 -9.53648821e+02 -3.70699535e+01 -3.50380279e+01
  7.01557491e+00  8.89965163e+01 -3.36699567e+01 -3.46404999e+02
 -2.66006107e+02 -3.41665595e+02 -5.37045270e+02 -5.99441487e+02
 -3.82356787e+01 -

12900
s = (array([73], dtype=int64),)
Gap = 0.38079553191300874
alpha = 0.0001550147263990079
criterion = 0.00013173133617456697
grad = [-4.14835392e+02 -5.95315626e+02 -8.12152431e+02 -8.30600633e+02
 -9.08354196e+02 -2.59786936e+03 -2.37462486e+02 -1.11607129e+02
 -5.73359646e+01 -1.92884345e+02 -6.62605185e+01  8.88082940e+01
  8.99459476e+01 -6.14990274e+02 -7.89563674e+02 -6.94644645e+02
 -6.94670077e+02 -7.67389607e+02 -7.27760071e+02 -6.59171753e+02
 -1.95366727e+01 -9.66351507e+02 -6.15029600e+01 -3.73486688e+01
 -6.28702964e+01  2.60565156e+01  2.26434005e+00  4.43218797e+01
 -2.33817640e+02  5.19690912e+01  6.24612974e+00 -1.91053826e+01
  1.10228893e+01  7.67825025e+01 -4.32436663e+02 -9.66351507e+02
 -6.59171753e+02 -1.90442299e+02 -1.31285606e+03 -2.08882555e+03
 -3.62603536e+02 -9.53401426e+02 -3.70431840e+01 -3.49481937e+01
  7.01146441e+00  8.89988134e+01 -3.36539669e+01 -3.46404870e+02
 -2.65962618e+02 -3.41572697e+02 -5.36877993e+02 -5.99211893e+02
 -3.82670505e+01 -1

13200
s = (array([66], dtype=int64),)
Gap = 0.21913607657654666
alpha = 0.0001514921981517952
criterion = 0.0001588190131545047
grad = [-4.14983442e+02 -5.95379461e+02 -8.12275492e+02 -8.30768767e+02
 -9.08685208e+02 -2.59775972e+03 -2.37709852e+02 -1.11731676e+02
 -5.74018397e+01 -1.93090265e+02 -6.63767494e+01  8.88068651e+01
  8.99456249e+01 -6.15211141e+02 -7.89708816e+02 -6.94858252e+02
 -6.94831243e+02 -7.67482917e+02 -7.27920240e+02 -6.59372648e+02
 -1.95488985e+01 -9.66548880e+02 -6.15321958e+01 -3.73835663e+01
 -6.29183651e+01  2.60528766e+01  2.24989761e+00  4.43200573e+01
 -2.33911121e+02  5.19648730e+01  6.23610698e+00 -1.91257659e+01
  1.10191720e+01  7.67830017e+01 -4.32486251e+02 -9.66548880e+02
 -6.59372648e+02 -1.90525265e+02 -1.31315932e+03 -2.08913743e+03
 -3.62828206e+02 -9.53598997e+02 -3.70579811e+01 -3.50313530e+01
  7.01143842e+00  8.89968267e+01 -3.36680024e+01 -3.46411220e+02
 -2.65987359e+02 -3.41657231e+02 -5.37033828e+02 -5.99433149e+02
 -3.82314683e+01 -1.

13500
s = (array([75], dtype=int64),)
Gap = 0.39925878923793334
alpha = 0.00014812620352540364
criterion = 0.00010024789732883114
grad = [-4.14810714e+02 -5.95295823e+02 -8.12163883e+02 -8.30598196e+02
 -9.08419886e+02 -2.59784742e+03 -2.37486432e+02 -1.11634125e+02
 -5.73292594e+01 -1.92883715e+02 -6.62627363e+01  8.88078963e+01
  8.99460290e+01 -6.14993816e+02 -7.89562055e+02 -6.94665113e+02
 -6.94679234e+02 -7.67388282e+02 -7.27782998e+02 -6.59150657e+02
 -1.95319114e+01 -9.66328727e+02 -6.15167707e+01 -3.73463430e+01
 -6.28785912e+01  2.60522341e+01  2.26067052e+00  4.43214977e+01
 -2.33832139e+02  5.19636230e+01  6.25156000e+00 -1.91082834e+01
  1.10136828e+01  7.67813259e+01 -4.32450655e+02 -9.66328727e+02
 -6.59150657e+02 -1.90448268e+02 -1.31291019e+03 -2.08892542e+03
 -3.62596254e+02 -9.53345513e+02 -3.70346077e+01 -3.49500424e+01
  7.01159277e+00  8.89984087e+01 -3.36497299e+01 -3.46409718e+02
 -2.65944089e+02 -3.41571715e+02 -5.36904839e+02 -5.99242749e+02
 -3.82630594e+01 -

In [18]:
lp.print_stats()

Timer unit: 3.0047e-07 s

Total time: 38.1386 s
File: <ipython-input-9-ce1b4291fa34>
Function: frank_wolfe_poisson at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def frank_wolfe_poisson(W,
     2                                                                   y,
     3                                                                   lam,
     4                                                                   x_0=None,
     5                                                                   alpha_policy='standard',
     6                                                                   max_iter=50000,
     7                                                                   eps=0.001,
     8                                                                   print_every=100,
     9                                                                   dtype=np.float64,
    10                                        

In [ ]:
%%time
x, \
alpha_hist, \
Gap_hist, \
Q_hist, \
time_hist, \
grad_hist = frank_wolfe_poisson(W, 
                                y,
                                lam,
                                x_0=np.ones(n) / n,
                                eps=1e-4, 
                                max_iter=50000, 
                                alpha_policy='standard', 
                                print_every=100, 
                                debug_info=True)

## ICML

In [46]:
lp = LineProfiler()
lp_wrapper = lp(frank_wolfe_poisson)
icml_x, \
icml_alpha_hist, \
icml_Gap_hist, \
icml_Q_hist, \
icml_time_hist, \
icml_grad_hist = lp_wrapper(W, y, lam,x_0=np.ones(n) / n,
                       eps=1e-4, max_iter=50000, 
                       alpha_policy='icml', 
                       print_every=100, 
                       debug_info=True)

********* Algorithm starts *********
1
s = (array([75], dtype=int64),)
Gap = 215778.92897541937
alpha = 0.0007145004201937056
criterion = 0.000711590019419764
grad = [-5.08552577e+04 -4.60356346e+04 -5.34270049e+04 -4.98588139e+04
 -5.59768918e+04 -1.76891436e+05 -1.97216613e+04 -8.62701842e+03
 -7.39555139e+03 -1.62228481e+04 -1.00284745e+04 -1.87766660e+01
  3.09925648e+01 -5.12015474e+04 -5.12708571e+04 -5.12244880e+04
 -5.12292494e+04 -5.12274610e+04 -4.19553846e+04 -5.75014917e+04
 -9.23685908e+03 -8.25384028e+04 -4.43245024e+03 -8.29406513e+03
 -1.07702804e+04 -3.99171897e+03 -5.05906113e+03 -3.35037557e+03
 -1.34282195e+04 -1.23872722e+03 -7.32999420e+03 -3.16278491e+03
 -2.55648546e+03 -3.14845347e+02 -3.37096303e+04 -8.25384028e+04
 -5.75014917e+04 -1.91545689e+04 -6.32495092e+04 -1.17635089e+05
 -3.48062050e+04 -8.43436740e+04 -8.00029315e+03 -7.87474420e+03
 -3.22137382e+03 -9.17766660e+01 -7.14573271e+03 -3.18700514e+04
 -2.56197986e+04 -2.83722382e+04 -3.16165019e+04 -3.22

300
s = (array([82], dtype=int64),)
Gap = 12779.600250493408
alpha = 0.005132153013142344
criterion = 0.005073932311306499
grad = [-5.81797235e+03 -5.70699207e+03 -6.89875559e+03 -6.62766214e+03
 -7.35189666e+03 -2.25434545e+04 -2.56293292e+03 -1.18113344e+03
 -9.07019992e+02 -2.06511337e+03 -1.19892524e+03  7.69453735e+01
  8.43017195e+01 -6.36558705e+03 -6.60646387e+03 -6.49819315e+03
 -6.42586206e+03 -6.50717267e+03 -5.62778147e+03 -7.00996814e+03
 -1.01955888e+03 -1.01075032e+04 -6.43321542e+02 -9.84091342e+02
 -1.28401605e+03 -4.08021623e+02 -5.63691932e+02 -3.19004402e+02
 -1.87827781e+03 -9.21719709e+01 -7.90298600e+02 -4.36860642e+02
 -2.78683238e+02  3.24286926e+01 -4.07056529e+03 -1.01075032e+04
 -7.00996814e+03 -2.35833073e+03 -8.85691147e+03 -1.59482843e+04
 -4.19764739e+03 -1.00169849e+04 -8.90447292e+02 -8.93749216e+02
 -3.45326811e+02  6.96077379e+01 -8.61922864e+02 -4.02116736e+03
 -2.98680184e+03 -3.61025480e+03 -4.33461256e+03 -4.43625728e+03
 -1.20390583e+03 -1.87631

600
s = (array([75], dtype=int64),)
Gap = 750.3467252886926
alpha = 0.0033222591362126247
criterion = 0.0022484072445489943
grad = [-1.21721093e+03 -1.37459000e+03 -1.74902853e+03 -1.72401460e+03
 -1.90326041e+03 -5.65384972e+03 -6.01137320e+02 -2.82628445e+02
 -1.85545743e+02 -4.72052144e+02 -2.35386015e+02  8.70097926e+01
  8.91243531e+01 -1.48668745e+03 -1.67829736e+03 -1.58040107e+03
 -1.57232826e+03 -1.65039033e+03 -1.48555467e+03 -1.61480687e+03
 -1.70184232e+02 -2.35318679e+03 -1.55734779e+02 -1.80136412e+02
 -2.48117191e+02 -4.10673407e+01 -8.66139591e+01 -1.08913944e+01
 -4.89482582e+02  2.81766436e+01 -1.13904031e+02 -8.66424323e+01
 -3.69464655e+01  6.94447016e+01 -9.93549415e+02 -2.35318679e+03
 -1.61480687e+03 -5.18476938e+02 -2.48808446e+03 -4.29178060e+03
 -9.20128160e+02 -2.29967229e+03 -1.60089868e+02 -1.55399546e+02
 -4.67747481e+01  8.61874125e+01 -1.59573864e+02 -9.31549993e+02
 -6.65807516e+02 -8.41018156e+02 -1.12746305e+03 -1.18865867e+03
 -2.21574946e+02 -4.1986

900
s = (array([82], dtype=int64),)
Gap = 211.23222181181222
alpha = 0.0022172949002217295
criterion = 0.0021764225199873274
grad = [-7.20035201e+02 -9.03487220e+02 -1.18771153e+03 -1.18916567e+03
 -1.30653346e+03 -3.81190619e+03 -3.85328360e+02 -1.82406482e+02
 -1.08299734e+02 -2.99991530e+02 -1.31572565e+02  8.81077624e+01
  8.96254063e+01 -9.57824719e+02 -1.14263155e+03 -1.04331389e+03
 -1.04186100e+03 -1.12130192e+03 -1.03201544e+03 -1.03115348e+03
 -7.85246631e+01 -1.51446116e+03 -1.01467898e+02 -9.29004571e+01
 -1.35290686e+02 -7.12205218e-01 -3.35560970e+01  2.24857731e+01
 -3.36424754e+02  4.25340721e+01 -4.10954140e+01 -4.74150408e+01
 -8.30181443e+00  7.38228574e+01 -6.54910829e+02 -1.51446116e+03
 -1.03115348e+03 -3.18414478e+02 -1.78799313e+03 -3.01035520e+03
 -5.65833522e+02 -1.46744012e+03 -8.19825910e+01 -7.52985907e+01
 -1.35687507e+01  8.79923674e+01 -8.32181131e+01 -5.92973265e+02
 -4.15286965e+02 -5.39141193e+02 -7.75409276e+02 -8.31694278e+02
 -1.14803023e+02 -2.610

1200
s = (array([66], dtype=int64),)
Gap = 98.58358650687751
alpha = 0.0016638935108153079
criterion = 0.0017488236257039535
grad = [-5.65026272e+02 -7.55056350e+02 -1.01007825e+03 -1.01970772e+03
 -1.11804444e+03 -3.23231280e+03 -3.16683964e+02 -1.50506634e+02
 -8.34584792e+01 -2.45653251e+02 -9.87142759e+01  8.84540106e+01
  8.97869083e+01 -7.91036159e+02 -9.73428039e+02 -8.74642533e+02
 -8.75040060e+02 -9.53766242e+02 -8.88714631e+02 -8.47549478e+02
 -4.96256397e+01 -1.24936127e+03 -8.40641577e+01 -6.54950705e+01
 -9.98706900e+01  1.19594116e+01 -1.69244470e+01  3.30210620e+01
 -2.88142060e+02  4.68319151e+01 -1.80618596e+01 -3.49509083e+01
  3.46791187e-01  7.51446696e+01 -5.48871435e+02 -1.24936127e+03
 -8.47549478e+02 -2.55589094e+02 -1.56654176e+03 -2.60043199e+03
 -4.56410699e+02 -1.20702971e+03 -5.76729109e+01 -5.11144227e+01
 -3.35133921e+00  8.85447064e+01 -5.91671298e+01 -4.84369456e+02
 -3.37442311e+02 -4.44090176e+02 -6.63982896e+02 -7.19613813e+02
 -8.05484026e+01 -2.108

...

9000
s = (array([75], dtype=int64),)
Gap = 1.8136075221718784
alpha = 0.00022217285047767163
criterion = 0.00015035149105762892
grad = [-4.17478806e+02 -5.98070635e+02 -8.15491028e+02 -8.33650996e+02
 -9.11618950e+02 -2.61025777e+03 -2.38275282e+02 -1.12028997e+02
 -5.75743954e+01 -1.93374007e+02 -6.65644208e+01  8.88044914e+01
  8.99445475e+01 -6.17837520e+02 -7.92622016e+02 -6.97681090e+02
 -6.97688445e+02 -7.70481345e+02 -7.30409376e+02 -6.62293952e+02
 -2.00570959e+01 -9.71108297e+02 -6.18423418e+01 -3.78150913e+01
 -6.35090994e+01  2.58116300e+01  1.95086347e+00  4.41285451e+01
 -2.34660134e+02  5.18729545e+01  5.83974030e+00 -1.93351164e+01
  1.08223036e+01  7.67507255e+01 -4.34443672e+02 -9.71108297e+02
 -6.62293952e+02 -1.91547525e+02 -1.31691697e+03 -2.09696015e+03
 -3.64246798e+02 -9.57820349e+02 -3.74174914e+01 -3.52369788e+01
  6.82703586e+00  8.89909843e+01 -3.41050724e+01 -3.48679228e+02
 -2.67302949e+02 -3.43367326e+02 -5.38958252e+02 -6.01109082e+02
 -3.90181454e+01 -1.

9300
s = (array([73], dtype=int64),)
Gap = 1.5914826753100897
alpha = 0.00021500752526338422
criterion = 0.00018271234391443593
grad = [-4.17358357e+02 -5.97934820e+02 -8.15297584e+02 -8.33501475e+02
 -9.11413555e+02 -2.60939083e+03 -2.38273046e+02 -1.12008656e+02
 -5.75943472e+01 -1.93403136e+02 -6.65796619e+01  8.88047480e+01
  8.99443678e+01 -6.17698523e+02 -7.92463874e+02 -6.97499777e+02
 -6.97522031e+02 -7.70321587e+02 -7.30249495e+02 -6.62158564e+02
 -2.00329489e+01 -9.70877805e+02 -6.18141216e+01 -3.77939007e+01
 -6.34660106e+01  2.58290783e+01  1.97000276e+00  4.41390864e+01
 -2.34608626e+02  5.18839952e+01  5.85553211e+00 -1.93231059e+01
  1.08437806e+01  7.67539845e+01 -4.34320827e+02 -9.70877805e+02
 -6.62158564e+02 -1.91483245e+02 -1.31666535e+03 -2.09644632e+03
 -3.64172540e+02 -9.57637903e+02 -3.74064109e+01 -3.52242445e+01
  6.83659138e+00  8.89917019e+01 -3.40820664e+01 -3.48545053e+02
 -2.67242132e+02 -3.43261976e+02 -5.38823066e+02 -6.00995189e+02
 -3.89740063e+01 -1.

9600
s = (array([75], dtype=int64),)
Gap = 1.7078133444160812
alpha = 0.00020828993959591752
criterion = 0.00014096046161796768
grad = [-4.17120761e+02 -5.97728398e+02 -8.15113192e+02 -8.33321199e+02
 -9.11290292e+02 -2.60847140e+03 -2.38266937e+02 -1.12019672e+02
 -5.75971547e+01 -1.93393263e+02 -6.65802259e+01  8.88043514e+01
  8.99443163e+01 -6.17533026e+02 -7.92277378e+02 -6.97302047e+02
 -6.97318425e+02 -7.70142966e+02 -7.30104424e+02 -6.61936850e+02
 -1.99969604e+01 -9.70584059e+02 -6.18142596e+01 -3.77595066e+01
 -6.34321989e+01  2.58415369e+01  1.99357179e+00  4.41492717e+01
 -2.34566541e+02  5.18951791e+01  5.88279200e+00 -1.93090940e+01
  1.08666649e+01  7.67577098e+01 -4.34173572e+02 -9.70584059e+02
 -6.61936850e+02 -1.91415039e+02 -1.31646432e+03 -2.09614055e+03
 -3.64012570e+02 -9.57267010e+02 -3.73733346e+01 -3.51836219e+01
  6.85783718e+00  8.89920774e+01 -3.40464867e+01 -3.48434597e+02
 -2.67115737e+02 -3.43135137e+02 -5.38721501e+02 -6.00889741e+02
 -3.89351107e+01 -1.

9900
s = (array([73], dtype=int64),)
Gap = 1.3631902736703978
alpha = 0.00020197939810139365
criterion = 0.000171639757341169
grad = [-4.16793182e+02 -5.97487683e+02 -8.14812669e+02 -8.33065863e+02
 -9.11010841e+02 -2.60734820e+03 -2.38264931e+02 -1.12027425e+02
 -5.75464095e+01 -1.93298410e+02 -6.65199558e+01  8.88051618e+01
  8.99448489e+01 -6.17200920e+02 -7.91977963e+02 -6.97032821e+02
 -6.97084022e+02 -7.69874511e+02 -7.29908091e+02 -6.61552294e+02
 -1.99325310e+01 -9.70028647e+02 -6.18023846e+01 -3.77057973e+01
 -6.33550419e+01  2.58566120e+01  1.99119894e+00  4.41706534e+01
 -2.34515365e+02  5.18749568e+01  5.93330545e+00 -1.93199783e+01
  1.08278419e+01  7.67519988e+01 -4.34089302e+02 -9.70028647e+02
 -6.61552294e+02 -1.91284173e+02 -1.31621582e+03 -2.09582687e+03
 -3.63703658e+02 -9.56652698e+02 -3.72848192e+01 -3.51061395e+01
  6.85680222e+00  8.89938112e+01 -3.40006674e+01 -3.48361792e+02
 -2.66911521e+02 -3.42968760e+02 -5.38567684e+02 -6.00738838e+02
 -3.88867227e+01 -1.54

10200
s = (array([75], dtype=int64),)
Gap = 1.6462497211053915
alpha = 0.00019603999215840032
criterion = 0.00013267582139391018
grad = [-4.16831336e+02 -5.97429562e+02 -8.14698469e+02 -8.32881827e+02
 -9.10769351e+02 -2.60773106e+03 -2.37959246e+02 -1.11881713e+02
 -5.74570448e+01 -1.93126761e+02 -6.64167261e+01  8.88059910e+01
  8.99453055e+01 -6.17088000e+02 -7.91869103e+02 -6.96941425e+02
 -6.96968358e+02 -7.69743659e+02 -7.29767590e+02 -6.61483647e+02
 -1.99314714e+01 -9.69920983e+02 -6.17594567e+01 -3.76937559e+01
 -6.33523025e+01  2.58619114e+01  2.01771522e+00  4.41725751e+01
 -2.34440654e+02  5.18828381e+01  5.94172458e+00 -1.92786884e+01
  1.08439771e+01  7.67539356e+01 -4.34020132e+02 -9.69920983e+02
 -6.61483647e+02 -1.91269392e+02 -1.31591639e+03 -2.09504413e+03
 -3.63788805e+02 -9.56720144e+02 -3.73190962e+01 -3.51451308e+01
  6.86017401e+00  8.89932542e+01 -3.39980066e+01 -3.48171240e+02
 -2.66996447e+02 -3.42941017e+02 -5.38443763e+02 -6.00612433e+02
 -3.88639352e+01 -1

In [47]:
lp.print_stats()

Timer unit: 3.0047e-07 s

Total time: 490.646 s
File: <ipython-input-26-f6b1651b93cd>
Function: frank_wolfe_poisson at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def frank_wolfe_poisson(W,
     2                                                                   y,
     3                                                                   lam,
     4                                                                   x_0=None,
     5                                                                   alpha_policy='standard',
     6                                                                   max_iter=50000,
     7                                                                   eps=0.001,
     8                                                                   print_every=100,
     9                                                                   dtype=np.float64,
    10                                       

In [ ]:
%%time
icml_x, \
icml_alpha_hist, \
icml_Gap_hist, \
icml_Q_hist, \
icml_time_hist, \
icml_grad_hist = frank_wolfe_poisson(W, 
                                y,
                                lam,
                                x_0=np.ones(n) / n,
                                eps=1e-4, 
                                max_iter=50000, 
                                alpha_policy='icml', 
                                print_every=100, 
                                debug_info=True)